In [1]:
import cv2
import numpy as np
import pathlib
import pandas as pd
import glob

In [ ]:
import cv2

def synchronize_and_display_videos_with_mapping(video_path1, video_path2, dataframe1, dataframe2, start_timestamp, stop_timestamp):
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)

    if not cap1.isOpened() or not cap2.isOpened():
        print("Error opening video file.")
        return

    # Find the starting frame numbers for each video based on the start timestamp
    start_frame1 = dataframe1[dataframe1['OE_timestamp'] == start_timestamp]['frame_number'].values[0]
    start_frame2 = dataframe2[dataframe2['OE_timestamp'] == start_timestamp]['frame_number'].values[0]

    # Set the starting frames for each video
    cap1.set(cv2.CAP_PROP_POS_FRAMES, start_frame1)
    cap2.set(cv2.CAP_PROP_POS_FRAMES, start_frame2)

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            break

        # Resize frames if needed (adjust the dimensions based on your preference)
        frame1 = cv2.resize(frame1, (640, 480))
        frame2 = cv2.resize(frame2, (640, 480))

        # Concatenate frames side by side
        synchronized_frame = cv2.hconcat([frame1, frame2])

        # Display the synchronized frame
        cv2.imshow('Synchronized Videos', synchronized_frame)

        # Check for the 'q' key to quit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

        # Find the next frame numbers based on the current timestamps
        current_timestamp = start_timestamp + (cap1.get(cv2.CAP_PROP_POS_FRAMES) - start_frame1) / cap1.get(cv2.CAP_PROP_FPS)
        next_frame1 = dataframe1[dataframe1['OE_timestamp'] == current_timestamp]['frame_number'].values[0]
        next_frame2 = dataframe2[dataframe2['OE_timestamp'] == current_timestamp]['frame_number'].values[0]

        # Set the next frames for each video
        cap1.set(cv2.CAP_PROP_POS_FRAMES, next_frame1)
        cap2.set(cv2.CAP_PROP_POS_FRAMES, next_frame2)

        # Update the start timestamp for the next iteration
        start_timestamp = current_timestamp

        # Break if the stop timestamp is reached
        if current_timestamp >= stop_timestamp:
            break

    # Release video capture objects and close the window
    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()

# Example usage:
# synchronize_and_display_videos_with_mapping('path_to_video1.mp4', 'path_to_video2.mp4', dataframe1, dataframe2, start_timestamp, stop_timestamp)


In [5]:
# define the path to the target block
block_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_24\2021_12_27\block_12')

# extract paths to objects of interest
le_video = [vid for vid in glob.glob(str(block_path) + r'\eye_videos\LE\**\*.mp4') if
                  "DLC" not in vid]
re_video = [vid for vid in glob.glob(str(block_path) + r'\eye_videos\RE\**\*.mp4') if
                  "DLC" not in vid]
le_analysis_path = block_path / 'analysis' / 'le_df.csv'
re_analysis_path = block_path / 'analysis' / 're_df.csv'

In [6]:
# This cell reads the analysis file and gets it ready for ellipse fitting in the video
r_df = pd.read_csv(re_analysis_path)
r_width = np.floor(r_df.width.values)

r_height = np.floor(r_df.height.values)

MmAxes_r = []
angle_r = np.floor(np.degrees(r_df.phi.values))

center_coords_r = [np.floor(r_df.center_x.values), np.floor(r_df.center_y.values)]

for i in range(len(r_width)):
    if r_width[i] == r_width[i]:
        if r_width[i] > r_height[i]:
            MmAxes_r.append((int(r_width[i]),int(r_height[i])))
        else:
            MmAxes_r.append((int(r_height[i]), int(r_width[i])))
    else:
        MmAxes_r.append((np.nan, np.nan))

In [7]:
### working ###
# This cell runs a video of the right eye with overlaid ellipses

re_cap = cv2.VideoCapture(re_video[0])
frame_n = -1
# go over frames of the video
while re_cap.isOpened():
    bad_flag = 0
    # get a frame
    re_ret, re_frame = re_cap.read()
    if re_ret is True:
        frame_n += 1
        # find the row in the dataframe that corresponds with the frame_n
        try:
            i = np.where(r_df.R_eye_frame.values.astype(np.int32) == frame_n)[0][0]
        except IndexError:
            i = 0

        re_frame = cv2.cvtColor(re_frame, cv2.COLOR_BGR2GRAY)
        if MmAxes_r[i][0] == MmAxes_r[i][0]:
            re_frame_with_ellipse = cv2.ellipse(re_frame,
                                                (int(center_coords_r[0][i]),int(center_coords_r[1][i])),
                                                (MmAxes_r[i][0], MmAxes_r[i][1]),
                                                angle_r[i],
                                                0,
                                                360,
                                                (0, 255, 0),
                                                2)
            cv2.imshow('Right eye frame', re_frame_with_ellipse)
        else:
            cv2.imshow('Right eye frame', re_frame)
        if cv2.waitKey(5) & 0xFF==ord('q'):
            break
    else:
        break
re_cap.release()

cv2.destroyAllWindows()

In [46]:
cv2.destroyAllWindows()